1. sample_persona1 : information (situation awareness, feasible action set) => classify , explanation
2. sample_persona2 : information + 각 persona의 분석 => classify , explanation
3. sample_persona3 : information + 각 persona의 분석 + persona간의 대화 => classify , explanation

In [1]:
import json
import openai
import time
import os
import random

openai.api_key = ''
f = open('agument.json')

env = json.load(f)
print('size of env : ', len(env))

size of env :  5129


In [2]:
prompt_robot = {'cooking' : '''### A list of feasible actions that a cooking robot can do is as follows.
feasible_actions = [grab, heat, plate]
### Additional description of the cooking robot is as follows.
1. The cooking robot can only handle detected objects.
2. The cooking robot can cook when an object is specified. 
3. The cooking robot does not operate if an objects are not specified.
4. The cooking robot does not perform actions not in the feasible_actions.
''',
'cleaning' : '''### A list of feasible actions that a cleaning robot can do is as follows.
feasible_actions = [clean, wipe]
### Additional description of the cleaning robot is as follows.
1. The cleaning robot can only clean detected objects.
2. The cleaning robot can clean scene in the workspaces.
3. The cleaning robot does not perform actions not in the feasible_actions.
''',
'massaging' : '''### A list of feasible actions that a massaging robot can do is as follows.
feasible_actions = [go_to, give_massage]
### Additional description of the massaging robot is as follows.
1. The massaging robot perform massage to the person in detected people.
2. The massaging robot cannot handle the object in the list of objects.
3. The massaging robot does not perform actions not in the feasible_actions.
'''
}

prompt_system = '''### According to what criteria are goals classified as certain, ambiguous, and infeasible?
1. If every objects mentioned in the goal exists and the robot can perform every actions mentioned in the goal, then the goal is certain.
2. If the goal leads the robot to process an object not in the detected objects, the goal is infeasible.
3. If the goal makes the robot do odd tasks unrelated to the robot, the goal is infeasible.
4. If the goal is a goal for another robot, the goal is infeasible.
5. If the goal does not specify a specific person, the goal is ambiguous. 
6. If the goal does not specify a specific object, the goal is ambiguous. 
'''

def prompt4persona4(sample):
    scene = sample['scene']
    goal = sample['goal']
    label = sample['label']
    task = sample['task']

    floorplan = scene['floorplan']
    objects = scene['objects']
    people = scene['people']

    prompt = f"### Look at the information below in perspective of the manager of the robot.\n\n"
    prompt += f"""situation awareness: ```
# Workspace where the robot can operate are as follows.
workspaces = {floorplan}
# Information about what objects and people are on workspaces is as follows. This information is the result of recognition by the robot.
objects = {objects}
people = {people}
```

"""
    prompt += f'''goal for the {task} robot:"""
{goal}
"""

'''

    return prompt

In [3]:
import multiprocessing
multiprocessing.cpu_count()

24

In [9]:
PATH_SAMPLE_READ = "data/explanation_thread.json"
PATH_SAMPLE_WRITE = "data/explanation_thread.json"
import os

samples = {}

if os.path.isfile(PATH_SAMPLE_READ):
    # checks if file exists
    print ("File exists and is readable")
    f = open(PATH_SAMPLE_READ)
    samples = json.load(f)
else:
    print ("File doesn't exist and generate samples")
    samples = {}

f = open('agument.json')
env = json.load(f)

def generate_explanation(env,sample_name):
    sample = env[sample_name].copy()
    label = sample['label']
    task = sample['task']
    if label == 0:
        str_label = 'certain'
    if label == 1:
        str_label = 'ambiguous'
    if label == 2:
        str_label = 'infeasible'

    messages = []
    # for _task in prompt_robot:
    messages.append({"role": "system", "content": prompt_system})
    messages.append({"role": "assistant", "content": prompt4persona4(sample)})
    for _task in prompt_robot:
        messages.append({"role": "system", "content": prompt_robot[_task]})
        
    persona_setting = {'Kelly' : f"Kelly's job is to insist that the goal is certain or not for the robot, based on the situation awareness and the feasible_actions.",
                   'Amy' : f"Amy's job is to insist that the goal is infeasible or not for the robot, based on the situation awareness and the feasible_actions.",
                   'Bank' : f"Bank's job is to insist that the goal is ambiguous or not for the robot, based on the situation awareness and the feasible_actions."}

    for persona in persona_setting:
        temp_messages = messages.copy()
        temp_messages.append({"role": "assistant", "content": persona_setting[persona]})
        temp_messages.append({"role": "assistant", "content": f"### Generate {persona}'s insistence for the goal of the {task} robot in 3 sentence:"})
        
        response = None
        
        for i in range(3):
            try:
                time.sleep(0.2)
                response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=messages,
                temperature = 0.8
                )
                break
            except:
                print('error')
                if i == 2:
                    return
                pass

        ans = response['choices'][0]['message']['content']
        sample[f"{persona}'s analysis"] = ans

    messages.append({"role": "assistant", "content": f"Kelly, Amy, and Bank each look at the above information and analyze the goal for the {task} robot. And the analysis are as follows."})

    for persona in persona_setting:
        messages.append({"role": "assistant", "content": f"{persona}'s analysis:\n"+sample[f"{persona}'s analysis"]})
        
    label = sample['label']
    messages.append({"role": "user", "content": f'### Classify the goal for {task} robot as certain, ambiguous, or infeasible. The response should be one word:\n{str_label}'})
    messages.append({"role": "user", "content": f'\nExplain why the goal for the {task} robot is {str_label} considering the situation awareness and the feasible actions of in 1 sentence:'})

    response = None
    for i in range(3):
        try:
            time.sleep(0.2)
            response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature = 0.8
            )
            break
        except:
            print('error')
            if i == 2:
                return
            pass

    ans = response['choices'][0]['message']['content']
    sample['explanation'] = ans
    env[sample_name] = sample
    return sample

samples_key_list = list(samples.keys())

import threading
import multiprocessing

max_num_proc = 20

for i in range(100):
    gen_exp_key_list = []
    for key in samples_key_list:
        if len(gen_exp_key_list) == max_num_proc:
            break
        sample = samples[key]
        if 'explanation' not in sample:
            gen_exp_key_list.append(key)

    print(gen_exp_key_list)

    process_list = []
    
    for i in range(len(gen_exp_key_list)):
        p = threading.Thread(target=generate_explanation, args = (samples,gen_exp_key_list[i]))
        p.start()
        process_list.append(p)

    for process in process_list:
        process.join(timeout=60)

    with open(PATH_SAMPLE_WRITE, "w") as outfile:
        json.dump(samples, outfile, indent=4)




File exists and is readable
['5087', '5096', '5101', '5102', '5106', '5113', '5114', '5117', '5119', '5120', '5122', '5123', '5125', '5126', '5127', '5128']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [5]:
# PATH_SAMPLE_READ = "data/explanation_thread.json"
# PATH_SAMPLE_WRITE = "data/explanation_thread.json"
# import os

# f = open(PATH_SAMPLE_READ)
# samples = json.load(f)


# f = open('agument.json')
# env = json.load(f)

# env_key_list = list(env.keys())

# for key in env_key_list:
#     if env[key]['label'] != 3:
#         if key not in samples:
#             samples[key] = env[key]

In [6]:
# with open(PATH_SAMPLE_WRITE, "w") as outfile:
#     json.dump(samples, outfile, indent=4)

error
